In [1]:
import shutil
import os

os.environ['KAGGLE_USERNAME'] = "bqhuy26" # username from the json file
os.environ['KAGGLE_KEY'] = "f3a6563216ae5ce0488297cb5b2f8d51" # key from the json fil

dataset_identifier = 'bqhuy26/gpr2000ae'
# !kaggle datasets download -d $dataset_identifier
!kaggle datasets download -d bqhuy26/gpr2000ae
zip_file_path = f'/kaggle/working/{dataset_identifier.split("/")[-1]}.zip'
shutil.unpack_archive(zip_file_path, '/kaggle/working')
import os
os.remove(zip_file_path)
image_size = 128

import os





100%|██████████████████████████████████████▉| 7.40G/7.41G [01:21<00:00, 103MB/s]
100%|██████████████████████████████████████| 7.41G/7.41G [01:21<00:00, 98.2MB/s]


In [2]:
import shutil
import os

# List of folders to remove
folders_to_remove = ['mask/defect','gprmaxdataset', 'gpr-filler', 'gpr-340-80', '900to980', 'gpr-420-80', 'gpr-260to339']

# List of files to remove
files_to_remove = ['src_only.in', 'unet_trained_model-Aug-5.h5', 'src_only.out']

# Remove folders
for folder in folders_to_remove:
    folder_path = os.path.join('/kaggle/working', folder)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f'Removed folder: {folder_path}')
    else:
        print(f'Folder not found: {folder_path}')

# Remove files
for file_name in files_to_remove:
    file_path = os.path.join('/kaggle/working', file_name)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f'Removed file: {file_path}')
    else:
        print(f'File not found: {file_path}')


Folder not found: /kaggle/working/mask/defect
Removed folder: /kaggle/working/gprmaxdataset
Removed folder: /kaggle/working/gpr-filler
Removed folder: /kaggle/working/gpr-340-80
Removed folder: /kaggle/working/900to980
Removed folder: /kaggle/working/gpr-420-80
Removed folder: /kaggle/working/gpr-260to339
Removed file: /kaggle/working/src_only.in
Removed file: /kaggle/working/unet_trained_model-Aug-5.h5
Removed file: /kaggle/working/src_only.out


In [3]:
folder_path = '/kaggle/working'
prefix_to_remove = '26pred'

for filename in os.listdir(folder_path):
    if filename.startswith(prefix_to_remove):
        file_path = os.path.join(folder_path, filename)
        os.remove(file_path)
        print(f'Removed file: {filename}')

print('All files with prefix "26pred" removed.')


Removed file: 26pred_14.png
Removed file: 26pred_13.png
Removed file: 26pred_3.png
Removed file: 26pred_10.png
Removed file: 26pred_11.png
Removed file: 26pred_1.png
Removed file: 26pred_2.png
Removed file: 26pred_9.png
Removed file: 26pred_15.png
Removed file: 26pred_7.png
Removed file: 26pred_12.png
Removed file: 26pred_19.png
Removed file: 26pred_16.png
Removed file: 26pred_5.png
Removed file: 26pred_17.png
Removed file: 26pred_8.png
Removed file: 26pred_6.png
Removed file: 26pred_4.png
Removed file: 26pred_18.png
All files with prefix "26pred" removed.


In [4]:
import cv2
import os
import numpy as np

def detect_colors(image, color_ranges):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    masks = []

    for color_range in color_ranges:
        lower_bound, upper_bound = color_range
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        masks.append(mask)

    return masks

def save_masked_images(image_dir, mask_dir, file_prefix, num_images, color_ranges):
    os.makedirs(mask_dir, exist_ok=True)

    for idx in range(1, num_images + 1):
        image_path = f'{image_dir}/{file_prefix}{idx}.png'
        image = cv2.imread(image_path)

        masks = detect_colors(image, color_ranges)
        
        cavity_mask = masks[0]
        healthy_mask = masks[1]
        
        # Combine both masks by logical OR
        combined_mask = cv2.bitwise_or(cavity_mask, healthy_mask)
        
        # Apply dilation to the combined mask to smooth out contours
        kernel = np.ones((5, 5), np.uint8)  # Define a dilation kernel
        dilated_mask = cv2.dilate(combined_mask, kernel, iterations=1)
        
        mask_filename = f'{file_prefix}{idx}_combined_mask.png'
        mask_file_path = os.path.join(mask_dir, mask_filename)
        cv2.imwrite(mask_file_path, dilated_mask)

        print(f'Saved combined mask for {file_prefix}{idx} as {mask_filename}')

        # Save cavity mask
        cavity_mask_filename = f'{file_prefix}{idx}_cavity_mask.png'
        cavity_mask_file_path = os.path.join(mask_dir, cavity_mask_filename)
        cv2.imwrite(cavity_mask_file_path, cavity_mask)
        print(f'Saved cavity mask for {file_prefix}{idx} as {cavity_mask_filename}')

        print(f'Processed image: {file_prefix}{idx}')

    print(f'Done processing {file_prefix} images')


image_dir = '/kaggle/working/image/defect'  # Set the path to your image directory
mask_dir = '/kaggle/working/mask/defect'  # Set the path to save the masked images
num_images = 999  # Number of images to process

# Define color ranges for red and yellow (in HSV format)
red_range = [(0, 100, 100), (10, 255, 255)]  # Red color range
yellow_range = [(20, 100, 100), (30, 255, 255)]  # Yellow color range

color_ranges = [red_range, yellow_range]

file_prefix = 'defect'  # Prefix of the input image files
save_masked_images(image_dir, mask_dir, file_prefix, num_images, color_ranges)

print('Processed all images')


Saved combined mask for defect1 as defect1_combined_mask.png
Saved cavity mask for defect1 as defect1_cavity_mask.png
Processed image: defect1
Saved combined mask for defect2 as defect2_combined_mask.png
Saved cavity mask for defect2 as defect2_cavity_mask.png
Processed image: defect2
Saved combined mask for defect3 as defect3_combined_mask.png
Saved cavity mask for defect3 as defect3_cavity_mask.png
Processed image: defect3
Saved combined mask for defect4 as defect4_combined_mask.png
Saved cavity mask for defect4 as defect4_cavity_mask.png
Processed image: defect4
Saved combined mask for defect5 as defect5_combined_mask.png
Saved cavity mask for defect5 as defect5_cavity_mask.png
Processed image: defect5
Saved combined mask for defect6 as defect6_combined_mask.png
Saved cavity mask for defect6 as defect6_cavity_mask.png
Processed image: defect6
Saved combined mask for defect7 as defect7_combined_mask.png
Saved cavity mask for defect7 as defect7_cavity_mask.png
Processed image: defect7

In [5]:
import cv2
import os
import numpy as np

def detect_colors(image, color_ranges):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    masks = []

    for color_range in color_ranges:
        lower_bound, upper_bound = color_range
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        masks.append(mask)

    return masks

def save_masked_images(image_dir, mask_dir, file_prefix, num_images, color_ranges):
    os.makedirs(mask_dir, exist_ok=True)

    for idx in range(1, num_images + 1):
        image_path = f'{image_dir}/{file_prefix}{idx}.png'
        image = cv2.imread(image_path)

        masks = detect_colors(image, color_ranges)
        
        cavity_mask = masks[0]
        healthy_mask = masks[1]
        
        # Combine both masks by logical OR
        combined_mask = cv2.bitwise_or(cavity_mask, healthy_mask)
        
        # Apply dilation to the combined mask to smooth out contours
        kernel = np.ones((5, 5), np.uint8)  # Define a dilation kernel
        dilated_mask = cv2.dilate(combined_mask, kernel, iterations=1)
        
        mask_filename = f'{file_prefix}{idx}_combined_mask.png'
        mask_file_path = os.path.join(mask_dir, mask_filename)
        cv2.imwrite(mask_file_path, dilated_mask)

        print(f'Saved combined mask for {file_prefix}{idx} as {mask_filename}')

        # Save cavity mask
        cavity_mask_filename = f'{file_prefix}{idx}_cavity_mask.png'
        cavity_mask_file_path = os.path.join(mask_dir, cavity_mask_filename)
        cv2.imwrite(cavity_mask_file_path, cavity_mask)
        print(f'Saved cavity mask for {file_prefix}{idx} as {cavity_mask_filename}')

        print(f'Processed image: {file_prefix}{idx}')

    print(f'Done processing {file_prefix} images')


image_dir = '/kaggle/working/image/healthy'  # Set the path to your image directory
mask_dir = '/kaggle/working/mask/healthy'  # Set the path to save the masked images
num_images = 999  # Number of images to process

# Define color ranges for red and yellow (in HSV format)
red_range = [(0, 100, 100), (10, 255, 255)]  # Red color range
yellow_range = [(20, 100, 100), (30, 255, 255)]  # Yellow color range

color_ranges = [red_range, yellow_range]

file_prefix = 'healthy'  # Prefix of the input image files
save_masked_images(image_dir, mask_dir, file_prefix, num_images, color_ranges)

print('Processed all images')


Saved combined mask for healthy1 as healthy1_combined_mask.png
Saved cavity mask for healthy1 as healthy1_cavity_mask.png
Processed image: healthy1
Saved combined mask for healthy2 as healthy2_combined_mask.png
Saved cavity mask for healthy2 as healthy2_cavity_mask.png
Processed image: healthy2
Saved combined mask for healthy3 as healthy3_combined_mask.png
Saved cavity mask for healthy3 as healthy3_cavity_mask.png
Processed image: healthy3
Saved combined mask for healthy4 as healthy4_combined_mask.png
Saved cavity mask for healthy4 as healthy4_cavity_mask.png
Processed image: healthy4
Saved combined mask for healthy5 as healthy5_combined_mask.png
Saved cavity mask for healthy5 as healthy5_cavity_mask.png
Processed image: healthy5
Saved combined mask for healthy6 as healthy6_combined_mask.png
Saved cavity mask for healthy6 as healthy6_cavity_mask.png
Processed image: healthy6
Saved combined mask for healthy7 as healthy7_combined_mask.png
Saved cavity mask for healthy7 as healthy7_cavity

## Testing